In [24]:
import pandas as pd
import os
import sys
from pathlib import Path
from prfndim.prfndim import Z, P, C, R, S
import numpy as np

In [23]:
path = Path.cwd().parent / "data" / "prfndim" / "raw_prfndim.csv"
df = pd.read_csv(path)

arities = []
for index, row in df.iterrows():
    arities.append(eval(row["expr"]).arity)
df["arity"] = arities
print(df)

                                                   expr  arity
0                                                   Z()    NaN
1                                                   S()    1.0
2                                               P(1, 1)    1.0
3                                               P(2, 1)    2.0
4                                               P(2, 2)    2.0
...                                                 ...    ...
9887                           R(P(1, 1), P(2, 1), Z())    1.0
9888                                    C(S(), P(4, 3))    4.0
9889              C(P(3, 3), P(4, 2), P(4, 4), P(4, 2))    4.0
9890  C(C(P(2, 1), Z(), P(2, 1)), Z(), C(P(2, 1), Z(...    2.0
9891  C(P(2, 2), C(C(P(2, 1), Z(), P(2, 1)), Z(), C(...    2.0

[9892 rows x 2 columns]


In [25]:
df_unique = df.drop_duplicates(subset=["expr"])
print(df_unique)

                                                   expr  arity
0                                                   Z()    NaN
1                                                   S()    1.0
2                                               P(1, 1)    1.0
3                                               P(2, 1)    2.0
4                                               P(2, 2)    2.0
...                                                 ...    ...
9877     C(C(P(3, 3), Z(), P(2, 2), P(2, 1)), Z(), Z())    NaN
9878  C(P(3, 3), P(4, 1), C(C(P(3, 3), Z(), P(2, 2),...    4.0
9881  C(P(3, 2), C(P(3, 2), P(2, 1), P(2, 1), P(2, 2...    2.0
9889              C(P(3, 3), P(4, 2), P(4, 4), P(4, 2))    4.0
9891  C(P(2, 2), C(C(P(2, 1), Z(), P(2, 1)), Z(), C(...    2.0

[4689 rows x 2 columns]


In [29]:
sample_max = 10
sample_num = 5
max_p_arity = 4

eq_domain = [np.zeros((1))] + [
    np.random.randint(1, sample_max + 1, size=(sample_num, dim))
    for dim in range(1, max_p_arity + 1)
]
eq_domain[1] = np.array(range(1, sample_max + 1)).reshape(-1, 1)

outputs_li = []
inputs_li = []
for index, row in df_unique.iterrows():
    expr = eval(row["expr"])
    if expr.arity is None:
        inputs = np.full((sample_num, 1), 0)
    else:
        inputs = eq_domain[expr.arity]
    outputs = tuple(expr.eval(*args) for args in inputs)
    inputs_li.append(tuple(tuple(int(arg) for arg in args) for args in inputs))
    outputs_li.append(tuple(int(output) for output in outputs))
df_unique["inputs"] = inputs_li
df_unique["outputs"] = outputs_li


df_unique_output = df_unique.drop_duplicates(subset=["arity", "outputs"])
df_unique_output = df_unique_output.sort_values(by="arity").reset_index(drop=True)
df_unique_output.to_csv("../data/prfndim/unique_output.csv", index=True)
print(df_unique_output)

                                                 expr  arity  \
0                                                 S()    1.0   
1                                             P(1, 1)    1.0   
2   R(C(P(2, 2), P(3, 3), P(3, 2)), P(4, 1), P(4, ...    1.0   
3               R(R(S(), P(2, 1), Z()), P(2, 2), Z())    1.0   
4   R(C(P(3, 2), Z(), P(3, 3), P(3, 2)), P(4, 2), ...    1.0   
..                                                ...    ...   
85  C(C(C(C(S(), P(3, 3)), P(3, 1), C(S(), P(3, 3)...    4.0   
86                                                Z()    NaN   
87  C(R(S(), C(P(3, 3), P(2, 1), P(2, 1), P(2, 2))...    NaN   
88                                C(S(), C(S(), Z()))    NaN   
89  C(C(C(P(2, 2), Z(), S()), P(2, 1)), C(C(P(2, 2...    NaN   

                                               inputs  \
0   ((1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,...   
1   ((1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,...   
2   ((1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,...   
3  

/tmp/ipykernel_12333/2993093469.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["inputs"] = inputs_li
/tmp/ipykernel_12333/2993093469.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["outputs"] = outputs_li
